## 144: Registro de Nuevos Usuarios con Validación y Seguridad

Hoy vas a implementar una ruta de registro de usuarios en tu API con FastAPI. Esto permitirá que nuevos usuarios:

📥 Se registren por su cuenta

🔐 Sean protegidos contra registros duplicados

🧂 Tengan sus contraseñas correctamente hasheadas

👥 Obtengan un rol por defecto (por ejemplo: usuario)

🧩 1. Agregar librerías necesarias (si aún no lo hiciste)
bash
Copiar
Editar
pip install sqlalchemy passlib
🧩 2. Ampliar tu modelo User en models.py (ya lo deberías tener)
Ya tienes esto definido:

python
Copiar
Editar
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

class User(Base):
    ...
    def verificar_password(self, password: str):
        return pwd_context.verify(password, self.hashed_password)

    @staticmethod
    def hashear_password(password: str):
        return pwd_context.hash(password)
🧩 3. Crear el modelo de solicitud para el registro
En main.py:

python
Copiar
Editar
from pydantic import BaseModel, EmailStr, Field

class RegistroRequest(BaseModel):
    username: str = Field(..., min_length=3)
    email: EmailStr
    password: str = Field(..., min_length=4)
🧩 4. Agregar la ruta /register en FastAPI
python
Copiar
Editar
from models import User, Base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine("sqlite:///usuarios.db")
Session = sessionmaker(bind=engine)
db = Session()

@app.post("/register")
def registrar_usuario(datos: RegistroRequest):
    # Verificar si el usuario o email ya existe
    if db.query(User).filter_by(username=datos.username).first():
        raise HTTPException(status_code=400, detail="Nombre de usuario ya registrado.")
    if db.query(User).filter_by(email=datos.email).first():
        raise HTTPException(status_code=400, detail="Correo ya registrado.")

    nuevo_usuario = User(
        username=datos.username,
        email=datos.email,
        hashed_password=User.hashear_password(datos.password),
        rol="usuario"  # Rol por defecto
    )
    db.add(nuevo_usuario)
    db.commit()

    return {"mensaje": "Registro exitoso", "usuario": datos.username}
🧪 5. Probar con cURL o Postman
bash
Copiar
Editar
curl -X POST http://localhost:8000/register \
-H "Content-Type: application/json" \
-d '{"username": "nuevo", "email": "nuevo@correo.com", "password": "1234"}'